# Workflow 1, Module 3, Question 1

## What toxicants does [condition] have specific sensitivity to?

### Expand service

In [3]:
robokop_server = 'robokop.renci.org'

In [4]:
import requests
import pandas as pd

In [5]:
def expand(type1,identifier,type2,rebuild=None,csv=None,predicate=None):
    url=f'http://{robokop_server}:80/api/simple/expand/{type1}/{identifier}/{type2}'
    params = {'rebuild': rebuild, 
              'csv'    : csv,
              'predicate': predicate} 
    params = { k:v for k,v in params.items() if v is not None }
    response = requests.get(url,params=params)
    print( f'Return Status: {response.status_code}' )
    if response.status_code == 200:
        return response.json()
    return []

In [6]:
def parse_answer(returnanswer):
    nodes = [answer['nodes'][1] for answer in returnanswer['answers']]
    edges = [answer['edges'][0] for answer in returnanswer['answers']]
    answers = [ {"result_id": node["id"], 
                 "result_name": node["name"] if 'name' in node else node['id'], 
                 "type": edge["type"],
                 "source": edge['edge_source']}
              for node,edge in zip(nodes,edges)]
    return pd.DataFrame(answers)

### Inputs

In [7]:
input_identifier='MONDO:0004979'
type1 = 'disease'
type2 = 'chemical_substance'

In [7]:
result = expand(type1,input_identifier,type2)
result_frame = parse_answer(result)
result_frame

Return Status: 200


,result_id,result_name,source,type
0,CHEBI:10100,zafirlukast,mychem.get_adverse_events,contributes_to
1,CHEBI:7492,nedocromil,mychem.get_adverse_events,contributes_to
2,CHEBI:93095,Heparin,ctd.disease_to_exposure,contributes_to
3,CHEBI:3001,Beclomethasone,mychem.get_adverse_events,contributes_to
4,CHEBI:3207,Budesonide,mychem.get_drugcentral,treats
5,CHEBI:27744,glyphosate,ctd.disease_to_exposure,contributes_to
6,CHEBI:28177,theophylline,mychem.get_drugcentral,treats
7,CHEBI:41879,Dexamethasone,ctd.disease_to_chemical,contributes_to
8,CHEBI:2659,Aminophylline,mychem.get_adverse_events,contributes_to
9,CHEBI:33101,Nitrogen Dioxide,ctd.disease_to_exposure,contributes_to


What are the different predicates we might be interested in?

In [13]:
result_frame['type'].value_counts()

contributes_to       134
treats                98
prevents              91
Unmapped_Relation     48
related_to            13
Name: type, dtype: int64

Here, contributes_to means that there are a lot of adverse events or that there is a study linking exposure to the disease, treats means that it's an indication, prevents means a small number of adverse events, Unmapped_Relation is a contra-indication, and related_to is coming from hmdb, probably meaning that this chemical is a metabolite associated with the disease.

So if you're looking for exogenous chemicals, `contributes_to` is probably the best, while if you're looking for endogenous, `related_to` is probably better.

There's a certain amount of care required though.  OFten, a chemical that treats X will also have a high number of adverse_events counts for X, so you may want to pull everything, and then do a little filtering to be careful.  This level of filtering is not shown here at the moment (TODO).

In [15]:
endogenous = result_frame[ result_frame['type'] == 'related_to' ]
endogenous

,result_id,result_name,source,type
15,CHEBI:68447,"1,3-dimethyluric acid",hmdb.metabolite_to_disease,related_to
26,CHEBI:28946,Theobromine,hmdb.metabolite_to_disease,related_to
38,CHEBI:68449,"1,7-dimethyluric acid",hmdb.metabolite_to_disease,related_to
39,CHEBI:691622,"1,3,7-trimethyluric acid",hmdb.metabolite_to_disease,related_to
82,CHEBI:27732,Caffeine,hmdb.metabolite_to_disease,related_to
95,CHEBI:62205,3-methylxanthine,hmdb.metabolite_to_disease,related_to
147,CHEBI:25858,"1,7-dimethylxanthine",hmdb.disease_to_metabolite,related_to
172,CHEBI:27595,11-epi-prostaglandin F2alpha,hmdb.metabolite_to_disease,related_to
174,CHEBI:62207,3-methyl-7H-xanthine,hmdb.metabolite_to_disease,related_to
335,CHEBI:68444,1-Methylxanthine,hmdb.disease_to_metabolite,related_to


In [16]:
exogenous = result_frame[ result_frame['type'] == 'contributes_to' ]
exogenous

,result_id,result_name,source,type
0,CHEBI:10100,zafirlukast,mychem.get_adverse_events,contributes_to
1,CHEBI:7492,nedocromil,mychem.get_adverse_events,contributes_to
2,CHEBI:93095,Heparin,ctd.disease_to_exposure,contributes_to
3,CHEBI:3001,Beclomethasone,mychem.get_adverse_events,contributes_to
5,CHEBI:27744,glyphosate,ctd.disease_to_exposure,contributes_to
7,CHEBI:41879,Dexamethasone,ctd.disease_to_chemical,contributes_to
8,CHEBI:2659,Aminophylline,mychem.get_adverse_events,contributes_to
9,CHEBI:33101,Nitrogen Dioxide,ctd.disease_to_exposure,contributes_to
10,CHEBI:34905,Paraquat,ctd.disease_to_exposure,contributes_to
11,CHEBI:6538,Loratadine,mychem.get_adverse_events,contributes_to


### Cystic Fibrosis

In [2]:
cf = 'MONDO:0009061'

In [8]:
result = expand(type1,cf,type2)
result_frame = parse_answer(result)
endogenous = result_frame[ result_frame['type'] == 'related_to' ]
endogenous

Return Status: 200


,result_id,result_name,source,type
4,CHEBI:31835,methylparaben,hmdb.metabolite_to_disease,related_to
5,CHEBI:16325,Lithocholic Acid,hmdb.metabolite_to_disease,related_to
6,CHEBI:30805,Dodecanoic acid,hmdb.metabolite_to_disease,related_to
7,CHEBI:30813,Capric acid,hmdb.metabolite_to_disease,related_to
8,CHEBI:17201,Glycyl-glycine,hmdb.metabolite_to_disease,related_to
10,CHEBI:28621,Triethanolamine,hmdb.metabolite_to_disease,related_to
12,CHEBI:16755,Chenodeoxycholic Acid,hmdb.metabolite_to_disease,related_to
13,CHEBI:28834,Deoxycholic Acid,hmdb.metabolite_to_disease,related_to
14,CHEBI:6650,Malic acid,hmdb.metabolite_to_disease,related_to
15,CHEBI:16359,Cholic acid,hmdb.metabolite_to_disease,related_to
